In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from torchinfo import summary
from scripts import data_setup, data_acquisition, engine, predict, plot_loss_curves
from pathlib import Path
from tqdm.auto import tqdm
from random import sample
from torch.utils.tensorboard import SummaryWriter
from IPython.display import Latex

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Setup Hyperparameters
BATCH_SIZE = 32
NUM_WORKERS = 0

In [ ]:
data_path = Path("data")
image_path = data_path / "pizza_steak_sushi_20%"
train_dir = image_path / "train"
test_dir = image_path /"test"

In [ ]:
# Setup manual transforms as required in the ViT

manual_transforms = transforms.Compose([
    transforms.Resize(size = (224,224)),
    transforms.ToTensor()
])

In [ ]:
# This code cell can be used to download a dataset from a URL and extract it to a specified directory

# This is a smaller version of the Food101 dataset

# data_acquisition.acquire_data(image_path=image_path,
#                               url = "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip",
#                               zip_name = "pizza_steak_sushi_20%")

# train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(data_dir=image_path,
#                                                                                train_transform=manual_transforms,
#                                                                                test_transform=manual_transforms,
#                                                                                batch_size=BATCH_SIZE,
#                                                                                num_workers=NUM_WORKERS)

In [ ]:
# Obtain required dataset. This may be commented out and the cell may be run again to download the dataset from a URL

train_dataset = datasets.CIFAR10(root = data_path,
                                  train = True,
                                  transform = manual_transforms,
                                  download = True)
test_dataset = datasets.CIFAR10(root = data_path,
                                train = False,
                                transform = manual_transforms,
                                download = True)

# Setup the dataloaders

train_dataloader = DataLoader(dataset = train_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=True,
                              num_workers=NUM_WORKERS)

test_dataloader = DataLoader(dataset = test_dataset,
                            batch_size=BATCH_SIZE,
                            shuffle=False,
                            num_workers=NUM_WORKERS)

class_names = train_dataset.classes

In [ ]:
# Inspect our data

print(f"Number of train images: {len(train_dataset)}, Number of test images: {len(test_dataset)}\n")
print(f"Number of train batches: {len(train_dataloader)}, Number of test batches: {len(test_dataloader)}\n")
print(f"{len(train_dataset) // BATCH_SIZE} batches in the train dataloader expected and {len(test_dataset) // BATCH_SIZE} batches in the test dataloader expected.\nNote: This may not be exactly equal and may differ by 1 batch due to rounding off\n")
print(f"Classes: {class_names}")

In [ ]:
# Visualizing a single image
first_image_batch, first_label_batch = next(iter(train_dataloader))
random_image_index = torch.randint(low = 0, high = len(first_image_batch), size=(1,)).item()

random_image = first_image_batch[random_image_index]
random_label = first_label_batch[random_image_index].item()

plt.figure(figsize = (12,8))
plt.imshow(random_image.permute(1,2,0))
plt.title(class_names[random_label])
plt.axis(False)
plt.show()

### Study of equations

#### Equation 1

\begin{align}
\mathbf{z}_0=[\mathbf{x}_{\mathrm{class}}; \mathbf{x}_p^1\mathbf{E}; \mathbf{x}_p^2\mathbf{E};\cdots; \mathbf{x}_p^N\mathbf{E}]+\mathbf{E}_{pos}
\end{align}

is the first equation. Here, we compute the input to the transformer encoder. 
\begin{align}
\mathbf{x}_{\mathrm{class}} 
\end{align}
is a learnable class token
\begin{align}
\mathbf{x}_p^i
\end{align}
is the i-th patch embedding obtained from the linear projection of the flattened image
\begin{align}
\mathbf{E}_{pos}^i
\end{align}
is the positional embedding corresponding to the i-th patch position


#### Equation 2

The second euqation describes the multi-head self attention (MSA). MSA mechanism applies self-attention multiple times in parallel.

Attention is when a token focues on another token and an attention head is a single instance of the self-attention mechanism.

First, 3 learned projection matrices are used to compute the matrices for Queires(Q), Keys(K) and Values(V), where each matrix is of an embedding dimension d Then, attention is calculated as
\begin{align}
Attention(Q,K,V) = softmax(\dfrac{QK^T}{\sqrt{d_k}})V
\end{align}

Softmax is used to make sure all attention scores are positive and sum up to 1. Attention scores are used to compute a weighted sum of the values V.

In MSA, we apply self-attention multiple times in parallel using different learned projection matrices for each head. This allows the model to capture different aspects of the relationship between tokens. Then, the attention heads are concatenated.

The second equation is,
\begin{align}
\mathbf{z^{\prime}}_\ell=\mathrm{MSA}(\mathrm{LN}(\mathbf{z}_{\ell-1}))+\mathbf{z}_{\ell-1}
\end{align}


#### Equation 3

The third equation describes the MLP part of the ViT. It is a feedforward neural network and it operates on each token independantly.
\begin{align}
z_{l+1} = MLP(LN(z^{\prime}_l)) + z^{\prime}_l
\end{align}

The activatoin function used here is GELU.

#### Equation 4

The final equation is the normalization of the previous layer. This summarizes the previous layers

### Replication

First we take a look at patch embeddings. If data can be represented in a learnable way, it is much easier to train a model. Here we use patches of size 16\*16. The number of patches depends on the original size (which is 224\*244\*3 here)

In [ ]:
height = 224
width = 224
color_channels = 3
patch_size = 16

number_of_patches = (height * width) // (patch_size ** 2)
print(f"Number of patches for image of size {height}x{width} with patch size {patch_size}: {number_of_patches}")

In [ ]:
embedding_layer_input_shape = (height, width, color_channels)
embegging_layer_output_shape = (number_of_patches, patch_size **2 * color_channels)

print(f"Input shape to embedding layer: {embedding_layer_input_shape}")
print(f"Output shape from embedding layer: {embegging_layer_output_shape}")

### Converting a single image into patches

In this section, we take the previous random image and convert it to patches, and in this example, a 224\*224 image to many images of size 16\*16

In [ ]:
# First, we create an array of zeros with the shape of the number of patches, patch size, patch size, and color channels.
# The patch size, patch size represents the height and width of the patch and the color channels represent the number of channels in the image (3 for RGB).
random_image_permuted = random_image.permute(1,2,0) # We permute the image to have the color channels at the end
rows = np.zeros ((height // patch_size, patch_size, width, color_channels))
for i in range(height // patch_size):
    row = random_image_permuted[i * patch_size: (i + 1) * patch_size, :,:]
    rows[i] = row
print(f"Shape of rows: {rows.shape}")

In [ ]:
fig, axes = plt.subplots(nrows = height // patch_size, 
                         ncols = 1, 
                         figsize = (12, 12))
fig.suptitle(f"{class_names[random_label]} broken down into rows")
for i,row in enumerate(rows):
    axes[i].imshow(row)
    axes[i].axis(False)
plt.show()

Above, we have seperated the rows into rows of height 1. Now, each row has to be seperated into patches of size 16\*16

In [ ]:
patches = np.zeros((number_of_patches, patch_size, patch_size, color_channels))
for i,row in enumerate(rows):
    for j in range(width // patch_size):
        patch = row[:, j * patch_size: (j + 1) * patch_size,:]
        patches[i * (width // patch_size) + j] = patch
print(patches[0].shape)

In [ ]:
fig, axes = plt.subplots(nrows = height // patch_size, 
                         ncols = width // patch_size, 
                         figsize = (12, 12))
fig.suptitle(f"{class_names[random_label]} broken down into patches")
for i in range(height // patch_size):
    for j in range(width // patch_size):
        axes[i,j].imshow(patches[i * width // patch_size + j])
        axes[i,j].axis(False)
plt.show()

### Creating patches with torch.nn.Conv2d

According to the ViT paper, 

`As an alternative to raw image patches, the input sequence can be formed from feature maps of a CNN (LeCun et al., 1989). In this hybrid model, the patch embedding projection E (Eq. 1) is applied to patches extracted from a CNN feature map. As a special case, the patches can have spatial size 1×1, which means that the input sequence is obtained by simply flattening the spatial dimensions of the feature map and projecting to the Transformer dimension. The classification input embedding and position embeddings are added as described above.`

Therefore, instead of just breaking the image into 16\*16 blocks, we can use a 2D convolutional layer to split the image and create a learnable embedding.

To create patches of required size, we set the kernel size to P and the stride also to P

In [ ]:
conv2d = nn.Conv2d(in_channels = 3, # 3 for RGB
                   out_channels = 768, # 768 is the number of hidden units in the ViT base model
                   kernel_size = patch_size, 
                   stride = patch_size,
                   padding = 1)


In [ ]:
image_after_convolution = conv2d(random_image.unsqueeze(0)) # We add a batch dimension
print(f"Shape of image after convolution: {image_after_convolution.shape}")

In [ ]:
random_indices = sample(population=range(768),
                        k = 5)
print(f"Random indices = {random_indices}")
fig, axes = plt.subplots(nrows = 1,
                         ncols = 5,
                         figsize = (20, 12))
squeezed_image = image_after_convolution.squeeze(dim = 0)
for i, index in enumerate(random_indices):
    axes[i].imshow(squeezed_image[index].detach().numpy())

### Flatten

Next, we need to flatten the learned embeddings. This needs to be done outside the number of out shannels dimensions. That is, we still want to make sure that the outermost shape is 768 as in the case of ViT base

In [ ]:
flattenLayer = nn.Flatten(start_dim = 2, 
                          end_dim = 3)
# Only the 14x14 patches are flattened

In [ ]:
flattened_patches = flattenLayer(image_after_convolution)

In [ ]:
print(f"Shape of flattened patches: {flattened_patches.shape}")
# We have successfully flattened the patches while maintaining the batch size and the number of patches

In [ ]:
random_indices = sample(population=range(768),
                        k = 5)
print(f"Random indices = {random_indices}")
fig, axes = plt.subplots(nrows = 5,
                         ncols = 1,
                         figsize = (20, 12))
for i, index in enumerate(random_indices):
    patch_plot = flattened_patches[0][index].unsqueeze(dim = 0) # 0 is used to index into the first 'batch'
    axes[i].imshow(patch_plot.detach().numpy())
    axes[i].axis(False)
    axes[i].set_title(f"Patch {index}")
plt.show()

The reason for flattening is because, originally, the transformer was designed to work with text and hence by transforming, we are essentially converting our 2D image into a 1D text representation

### Creating a class for creating patch embeddings

In this section, we create a class out of the code above. We let it inherit from nn.Module so that we can easily use it in a pytorch model

In [ ]:
class PatchEmbedding(nn.Module):
    """Creates a patch embedding layer. This takes in a 2D image and 
    converts it into a sequence of learnable patches.
    Args:
        in_channels (int): Number of input channels.
        patch_size (int): The height and width of each patch.
        emb_dim (int): The embedding dimension of the output patches."""
    
    def __init__(self,
                 in_channels = 3,
                 patch_size = 16,
                 out_channels = 768):
        super().__init__()
        self.patcher = nn.Conv2d(in_channels=in_channels,
                                 out_channels=out_channels,
                                 kernel_size=patch_size,
                                 stride=patch_size)
        
        self.flatten = nn.Flatten(start_dim=2, 
                                  end_dim=3)
    
    def forward(self,x):
        x_patched  = self.patcher(x)
        x_flattened = self.flatten(x_patched)
        return x_flattened.permute(0,2,1) # This is because the encoder expects (batch_size, num_patches, emb_dim)

In [ ]:
patchify = PatchEmbedding(in_channels = 3,
                            patch_size = 16,
                            out_channels = 768)
patchified_image = patchify(random_image.unsqueeze(0)) # We need to pass in the image with a batch dimension

print(f"Shape of patchified image: {patchified_image.shape}")

Upto this point, we have coded the patch embedding part of the vision transformer in the first equation. What's left is the position embedding

In [ ]:
summary(model = patchify,
        input_size = (32,3,224,224),
        col_names=["input_size", "output_size", "num_params","trainable"],
        row_settings = ["var_names"])

### Add class token embedding

In [ ]:
batch_size = patchified_image.shape[0]
embedding_dimension = patchified_image.shape[2]

class_token = nn.Parameter(torch.randn(batch_size, 1, embedding_dimension), requires_grad=True)
# (batch_size -> for each image im the batch, 1-> 1 class token, embedding_dimension->for each embedding)

print(f"Shape of class token: {class_token.shape}")

In [ ]:
patchified_image_with_class = torch.cat((class_token, patchified_image), dim = 1)
print(f"Shape of patchified image with class token: {patchified_image_with_class.shape}")

In [ ]:
position_embedding = nn.Parameter(torch.randn(1, number_of_patches + 1, embedding_dimension), requires_grad=True)
print(f"Shape of position embedding: {position_embedding.shape}")
position_and_patch_embedding  = patchified_image_with_class + position_embedding
print(f"Shape of position and patch embedding: {position_and_patch_embedding.shape}")

A brief summary: The input to the encoder contains:
 * patch embedding: This is a learnable embedding of size(batch_size, number_of_patches,embedding_dimension)
 * class token: This is a learnable class token of size(batch_size,1,embedding_dimension)
 * position embedding: This is a learnable embedding of size(batch_size, number_of_patches + 1, embedding_dimension)

First, the class token is concatenated with the patch embedding to produce a tensor of size (batch_size, number_of_patches + 1,embedding_dimension). The it's shape matches that of teh position embedding and can be added together


### Integrating all into a single code cell

In [ ]:
patch_size = 16

print(f"Random image shape: {random_image.shape}")

random_image_with_batch = random_image.unsqueeze(0)
print(f"Random image shape with batch: {random_image_with_batch.shape}")

patchify = PatchEmbedding(in_channels = 3,
                          patch_size = patch_size,
                          out_channels=768)

patch_embedded_image  = patchify(random_image_with_batch)
print(f"Shape of patch embedded image: {patch_embedded_image.shape}")

batch_size = patch_embedded_image.shape[0]
number_of_patches = patch_embedded_image.shape[1]
embedding_dimension = patch_embedded_image.shape[2]

class_token = nn.Parameter(torch.randn(batch_size, 1, embedding_dimension), requires_grad=True)
print(f"Shape of class token: {class_token.shape}")

patch_embedded_image_with_class = torch.cat((class_token, patch_embedded_image), dim = 1)
print(f"Shape of patch embedded image with class token: {patch_embedded_image_with_class.shape}")

position_embedding = nn.Parameter(torch.randn(1, number_of_patches + 1, embedding_dimension), requires_grad=True)
print(f"Shape of position embedding: {position_embedding.shape}")

position_and_patch_embedding = patch_embedded_image_with_class + position_embedding
print(f"Shape of position and patch embedding: {position_and_patch_embedding.shape}")

### Creating the MSA Block

In this section, we create the MSA block which is the main focus in equation 2 of the transformer equations.

In [ ]:
class MultiHeadSelfAttentionBlock(nn.Module):
    """Creates a single MultiHeadSelfAttentionBlock.
    Args:
        embedding_dimension (int): The embedding dimension of the input.
        num_heads (int): The number of heads in the multi-head attention block.
        attention_dropout (float): The dropout rate to apply to the attention scores.
        """
    def __init__(self,
                 embedding_dimension :int = 768, # ViT base has 768 embedding dimensions so we set 768 by default
                 num_heads : int = 12, # ViT base has 12 heads so we set 12 by default
                 attention_dropout : float = 0): # No dropout by default for ViT
        super().__init__()
        self.layer_norm = nn.LayerNorm(normalized_shape = embedding_dimension)
        self.multi_head_attention = nn.MultiheadAttention(embed_dim=embedding_dimension,
                                                          num_heads=num_heads,
                                                          dropout=attention_dropout)
    
    def forward(self,x):
        x = self.layer_norm(x)
        attn_output,_ = self.multi_head_attention(query = x,
                                                  key = x,
                                                  value = x,
                                                  need_weights=False)
        return attn_output


In [ ]:
msa_block = MultiHeadSelfAttentionBlock(embedding_dimension=768,
                                        num_heads=12,
                                        attention_dropout=0)

msa_output = msa_block(position_and_patch_embedding)
print(f"Shape of MultiHeadSelfAttentionBlock output: {msa_output.shape}")

### Equation 3: The MLP

In ViT, the MLP consists of 2 layers with GELU activation. In this section, we replicate this with layers and create a class

In [ ]:
class MLPBlock(nn.Module):
    """Creates a single MLP block with following structure:
    linear->GELU->dropout->linear->dropout
    Args:
    embedding_dimension (int): The embedding dimension of the input.
    mlp_size (int): size of each layer of MLP.
    mlp_dropout (float): dropout probability to be used in the dropout layers"""
    
    def __init__(self,
                 embedding_dimension:int = 768,
                 mlp_size: int = 3072,
                 mlp_dropout: float = 0.1):
        super().__init__()
        self.layer_norm = nn.LayerNorm(normalized_shape=embedding_dimension)
        self.mlp = nn.Sequential(
            nn.Linear(in_features = embedding_dimension,
                      out_features = mlp_size,
                      bias= True),
            nn.GELU(),
            nn.Dropout(p = mlp_dropout),
            nn.Linear(in_features= mlp_size,
                      out_features= embedding_dimension,
                      bias = True),
            nn.Dropout(p = mlp_dropout)
        )
    
    def forward(self,x):
        x = self.layer_norm(x)
        x = self.mlp(x)
        return x

In [ ]:
mlpblock = MLPBlock(embedding_dimension = 768,
                    mlp_size = 3072,
                    mlp_dropout = 0.1)

patched_image_after_mlp_block = mlpblock(msa_output)
print(f"Shape after MLP block: {patched_image_after_mlp_block.shape}")

### Creating the transformer encoder

An encoder is a stack of layers that converts an input into some form of numerical representation. According to the ViT paper,

`The Transformer encoder (Vaswani et al., 2017) consists of alternating layers of multiheaded selfattention (MSA, see Appendix A) and MLP blocks (Eq. 2, 3). Layernorm (LN) is applied before every block, and residual connections after every block (Wang et al., 2019; Baevski & Auli, 2019).`

MSA and MLP layers have been implemented above. What's left are the residual connections. These connections add back the input to the output of the MSA and MLP blocks.

Why add skip connections? Skip connections improve accuracy and generalization and also solve the vanishing gradient probelm.

In order to create the transformer encoder, we create a class of it and use the above created MSA and MLP blocks. The input to the transformer encoder are the patch and position embeddings along with the class token we created as part of equation 1.

In [ ]:
class TransformerEncoder(nn.Module):
    """Creates a transformer encoder block. It creates an MSA and MLP block
    Args:
    embedding_dimension (int): The embedding dimension of the input.
    num_heads (int): The number of heads in the multi-head attention block.
    attention_dropout (float): The dropout rate to apply to the attention scores.
    mlp_size (int): size of each layer of MLP.
    mlp_dropout (float): dropout probability to be used in the dropout layers
    """
    def __init__(self, # All default values are what are used in ViT base model
                 embedding_dimension: int = 768, 
                 num_heads: int = 12,
                 attention_dropout: float = 0.0,
                 mlp_size: int = 3072,
                 mlp_dropout: float = 0.1):
        super().__init__()
        self.msablock = MultiHeadSelfAttentionBlock(embedding_dimension = embedding_dimension,
                                                    num_heads = num_heads,
                                                    attention_dropout = attention_dropout)
        self.mlpblock = MLPBlock(embedding_dimension = embedding_dimension,
                                 mlp_size = mlp_size,
                                 mlp_dropout = mlp_dropout)
    
    def forward(self,x):
        x = self.msablock(x) + x
        x = self.mlpblock(x) + x
        # Above, weadd residual connections as stated in the ViT paper
        return x

In [ ]:
transformerencoder = TransformerEncoder(embedding_dimension = 768,
                                        num_heads = 12,
                                        attention_dropout = 0.0,
                                        mlp_size = 3072,
                                        mlp_dropout = 0.1)

print(summary(model = transformerencoder, 
              input_size = (1,197,768), 
              col_names=["input_size", "output_size", "num_params", "trainable"]))

An alternate method is to use pytorch's built in torch.nn.TransformerEncoderLayer(). This makes it easier to create a transformer encoder and also it may be less prone to errors and may even be faster in most cases

In [ ]:
torch_transformer_encoder = nn.TransformerEncoderLayer(d_model = 768,
                                                       nhead = 12,
                                                       dim_feedforward = 3072,
                                                       dropout = 0.1,
                                                       activation = "gelu",
                                                       batch_first = True,
                                                       norm_first = True)
print(torch_transformer_encoder)
print(summary(model = torch_transformer_encoder,
              input_size = (1,197,768), 
              col_names=["input_size", "output_size", "num_params", "trainable"],
              row_settings=["var_names"]))

In [ ]:
torch_transformer_encoder_layers = nn.TransformerEncoder(encoder_layer=torch_transformer_encoder,
                                                         num_layers = 12)
print(torch_transformer_encoder_layers)
print(summary(model = torch_transformer_encoder_layers,
              input_size = (1,197,768), 
              col_names=["input_size", "output_size", "num_params", "trainable"],
              row_settings=["var_names"]))

### Putting it all together

In this section, we combine all the previous steps and create the ViT. 

In [ ]:
class ViT(nn.Module):
    """Creates a Vision Transformer architecture with ViT-Base hyperparameters
    Args:
        img_size (int): Size of the input images (assumed to be square).
        in_channels (int): Number of input channels.
        patch_size (int): Size of the patches to split the image into.
        embedding_dimension (int): Dimensionality of the patch embeddings.
        num_heads (int): Number of attention heads.
        mlp_size (int): Hidden layer size in the MLP block.
        attn_dropout (float): Dropout rate for the attention layer.
        mlp_dropout (float): Dropout rate for the MLP block.
        embedding_dropout (float): Dropout rate for the patch embeddings.
        num_transformer_encoders (int): Number of transformer encoder layers.
        num_classes (int): Number of output classes.
    """
    def __init__(self,
                 img_size:int = 224,
                 in_channels:int = 3,
                 patch_size:int = 16,
                 embedding_dimension:int = 768,
                 num_heads:int = 12,
                 mlp_size:int = 3072,
                 attn_dropout:float = 0,
                 mlp_dropout:float = 0.1,
                 embedding_dropout:float = 0.1,
                 num_transformer_encoders:int = 12,
                 num_classes:int = 10
                 ):
        super().__init__()

        # Make sure image size is divisible by patch size
        assert img_size % patch_size == 0, f"Image size {img_size} not divisible by patch size {patch_size} "

        self.num_patches = (img_size * img_size) // patch_size**2

        # Create the patch embedding. This applies for every image in the batch
        self.patch_embedding = PatchEmbedding(in_channels = in_channels,
                                              patch_size=patch_size,
                                              out_channels = embedding_dimension)

        # Class token is for 1 image in the batch
        self.class_token = nn.Parameter(torch.randn(1, 1, embedding_dimension), requires_grad=True)

        # Similarly, position embedding is for 1 image in the batch
        self.position_embedding = nn.Parameter(torch.randn(1, self.num_patches + 1, embedding_dimension), requires_grad=True)

        self.embedding_dropout = nn.Dropout(p = embedding_dropout)

        self.transformer_encoders = nn.Sequential(*[TransformerEncoder(embedding_dimension = embedding_dimension,
                                                                       num_heads = num_heads,
                                                                       attention_dropout = attn_dropout,
                                                                       mlp_size = mlp_size,
                                                                       mlp_dropout = mlp_dropout) for _ in range(num_transformer_encoders)])
        
        self.classifier = nn.Sequential(nn.LayerNorm(normalized_shape = embedding_dimension),
                                        nn.Linear(in_features = embedding_dimension,
                                                  out_features = num_classes))
    
    def forward(self,x):
        x = self.patch_embedding(x)

        batch_size = x.shape[0]

        class_token = self.class_token.expand(batch_size, -1, -1)

        x = torch.cat((class_token, x), dim = 1)

        position_embedding = self.position_embedding.expand(batch_size, -1, -1)

        x += position_embedding

        x = self.embedding_dropout(x)

        x = self.transformer_encoders(x)

        x = self.classifier(x[:,0])

        return x
                

In [ ]:
# Example of creating the class embedding and expanding over a batch dimension
batch_size = 32
class_token_embedding_single = nn.Parameter(data=torch.randn(1, 1, 768)) # create a single learnable class token
class_token_embedding_expanded = class_token_embedding_single.expand(batch_size, -1, -1) # expand the single learnable class token across the batch dimension, "-1" means to "infer the dimension"

# Print out the change in shapes
print(f"Shape of class token embedding single: {class_token_embedding_single.shape}")
print(f"Shape of class token embedding expanded: {class_token_embedding_expanded.shape}")

In [ ]:
# random_image_tensor = torch.randn(16, 3, 224, 224).cuda() # (batch_size, color_channels, height, width)

# # Create an instance of ViT with the number of classes we're working with (pizza, steak, sushi)
# vit = ViT(num_classes=len(class_names)).to(device)

# # Pass the random image tensor to our ViT instance
# vit(random_image_tensor)

In [ ]:
# # Print a summary of our custom ViT model using torchinfo (uncomment for actual output)
# summary(model=vit,
#         input_size=(32, 3, 224, 224), # (batch_size, color_channels, height, width)
#         # col_names=["input_size"], # uncomment for smaller output
#         col_names=["input_size", "output_size", "num_params", "trainable"],
#         col_width=20,
#         row_settings=["var_names"]
# )

In [ ]:
# loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(params=vit.parameters(), 
#                              weight_decay=0.3,
#                              betas=(0.9,0.999),
#                              lr=3e-3)

In [ ]:
# history, all_preds, all_labels, writer = engine.train(model = vit,
#                                                     model_name="ViT",
#                                                     data_name="CIFAR10",
#                                                     epochs = 5,
#                                                     train_dataloader=train_dataloader,
#                                                     val_dataloader=test_dataloader,
#                                                     device=device,
#                                                     optimizer=optimizer,
#                                                     loss_fn=loss_fn,    
#                                                     )

### Utilizing Transfer Learning

In this section, we will use a pre existing ViT model and fine tune it on the CIFAR-10 dataset. We will use the ViT model from the torchvision library

In [ ]:
weights = torchvision.models.ViT_B_16_Weights.DEFAULT

pre_trained_vit = torchvision.models.vit_b_16(weights = weights).to(device)

summary(model = pre_trained_vit,
        input_size = (32, 3, 224, 224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        row_settings=["var_names"])

In [ ]:
for parameter in pre_trained_vit.parameters():
    parameter.requires_grad = False

summary(model = pre_trained_vit,
        input_size = (32, 3, 224, 224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        row_settings=["var_names"])

In [ ]:
pre_trained_vit.heads = nn.Linear(in_features = 768,
                                    out_features = 10).to(device)

In [ ]:
summary(model = pre_trained_vit,
        input_size = (32, 3, 224, 224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        row_settings=["var_names"])

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=pre_trained_vit.parameters(), 
                             weight_decay=0.3,
                             betas=(0.9,0.999),
                             lr=3e-3)

In [ ]:
history, all_preds, all_labels, writer = engine.train(model = pre_trained_vit,
                                                        model_name="ViT",
                                                        data_name="CIFAR10",
                                                        epochs = 5,
                                                        train_dataloader=train_dataloader,
                                                        val_dataloader=test_dataloader,
                                                        device=device,
                                                        optimizer=optimizer,
                                                        loss_fn=loss_fn)

### Plot Loss Curves

In [ ]:
plot_loss_curves.plot_loss_curves(history)